In [1]:
import json
import zipfile
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import math
import csv
import operator
import Geohash
from build import TreeBuilding
%matplotlib inline

In [2]:
air_trajs = pd.read_csv('air_trajs.csv',
                        #index_col=,
                        usecols = ['POLYLINE', 'Destination'],
                        converters={'POLYLINE': lambda x: json.loads(x), 'Destination': lambda x: json.loads(x)})

In [3]:
trajectories = list(air_trajs['POLYLINE'])
targets = list(air_trajs['Destination'])

In [4]:
tb = TreeBuilding()
tb = tb.build_tree(trajectories, targets, 1, 4000)

length 4328
('len', 4328)
len of inside point:  4218
len of inside without short trajectories points:  3756
len of outside points:  110
length 3756
length 110


In [11]:
a = [1,2,3,4,5,6,7,8,9]
[i for i in a if i >= 4]

[4, 5, 6, 7, 8, 9]

In [24]:
tb

<tree.Tree instance at 0x1074c8488>

In [8]:
air_trajs

,Destination,POLYLINE
0,"[-8.655561, 41.17131]","[[-8.669367, 41.235057], [-8.669196, 41.236461..."
1,"[-8.636688, 41.164461]","[[-8.669628, 41.234859], [-8.670348, 41.233932..."
2,"[-8.614818, 41.151078]","[[-8.669646, 41.237271], [-8.669628, 41.237226..."
3,"[-8.668224, 41.238657]","[[-8.670006, 41.2371], [-8.669727, 41.235696],..."
4,"[-8.669205, 41.236578]","[[-8.669259, 41.236668], [-8.669322, 41.236668..."
5,"[-8.590365, 41.16888]","[[-8.670015, 41.236965], [-8.669727, 41.235633..."
6,"[-8.669295, 41.241294]","[[-8.669295, 41.241303], [-8.669295, 41.241303..."
7,"[-8.6301, 41.158899]","[[-8.669583, 41.236254], [-8.669268, 41.236128..."
8,"[-8.613666, 41.123106]","[[-8.669295, 41.241285], [-8.669286, 41.241294..."
9,"[-8.606547, 41.153193]","[[-8.6706, 41.233761], [-8.671905, 41.23287], ..."
